## Bibliotecas

In [1]:
import torch
import torch.nn as nn

### Objetivo
Essa CNN é projetada para tarefas de classificação de imagens, onde a rede aprende a identificar padrões visuais (como bordas, formas e texturas) e, no final, classifica a imagem em uma das categorias possíveis (`num_classes`).

---

### Estrutura da Rede

#### 1. `nn.Conv2d`
Aplica filtros (ou kernels) nas imagens de entrada para detectar padrões como bordas, texturas, formas, etc.

**Parâmetros principais:**
- `in_channels=3`: número de canais da imagem de entrada (ex: 3 para RGB).
- `out_channels=64`: número de filtros que serão aprendidos — a rede irá gerar 64 "imagens" diferentes com padrões distintos.
- `kernel_size=11`: tamanho de cada filtro (nesse caso, 11x11).
- `stride=4`: quanto o filtro "anda" na imagem a cada passo. Um valor maior reduz mais rapidamente o tamanho da imagem.
- `padding=2`: adiciona bordas com zeros ao redor da imagem para preservar mais informações nas bordas.

#### 2. `nn.ReLU`
Aplica a função de ativação ReLU, que ajuda a rede a aprender relações não lineares (ou seja, mais complexas) entre os dados.

#### 3. `nn.MaxPool2d`
Reduz o tamanho da imagem ("compressa" a informação) pegando os valores mais altos de regiões locais. Isso ajuda a manter só as informações mais relevantes.

---

### Bloco `features`
Essa sequência aplica várias camadas de convolução, ativação e pooling. A ideia é:
- Aprender padrões simples nas primeiras camadas (ex: bordas, cores).
- Evoluir para padrões mais complexos nas camadas seguintes (ex: partes de objetos, rostos, etc).

---

### Camada `avgpool`

Reduz a imagem para um tamanho fixo (6x6), independente do tamanho que ela tinha antes. Isso é essencial para a etapa seguinte (camadas densas), que exige entrada de tamanho fixo.

---

### Camada `classifier`
Responsável pela classificação final com base nas características extraídas.

- Transforma os dados em um vetor de uma dimensão.
- Passa por camadas densas (`Linear`) com ReLU e Dropout.
- A última camada gera a saída com o número de classes desejado.

---

### Método `forward`
Esse método define o fluxo de dados pela rede:
1. Extração de características (features).
2. Redução de tamanho (avgpool).
3. Transformação em vetor (flatten).
4. Classificação final (classifier).

In [ ]:
class CNNNet(nn.Module):
    def __init__(self, num_classes=2): # construtor que monta a rede
        super(CNNNet, self).__init__() # faz com que a classe funcione como um modulo do pytorch, ativando as funções internas dele
        self.features == nn.Sequential(nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
                                       nn.ReLU(),
                                       nn.MaxPool2d(kernel_size=3, stride=2), # diminui o tamanho da imagem e mantém só as informações mais fortes
                                       nn.Conv2d(64, 192, kernel_size=5, padding=2),
                                       nn.ReLU(),
                                       nn.MaxPool2d(kernel_size=3, stride=2),
                                       nn.Conv2d(192, 384, kernel_size=3, padding=1),
                                       nn.ReLU(),
                                       nn.Conv2d(384, 256, kernel_size=3, padding=1),
                                       nn.ReLU(),
                                       nn.Conv2d(256, 256, kernel_size=3, padding=1),
                                       nn.ReLU(),
                                       nn.MaxPool2d(kernel_size=3, stride=2),)
        
        self.avgpool = nn.AdaptiveAvgPool2d((6,6)) # reduz o tamanho da imagem para um tamanho fixo: (6,6), independente de como ela chegou até aqui 
        # porque para a etapa de classificação a rede precisa de um tamanho fixo para funcionar
        self.classifier = nn.Sequential(
                          nn.Dropout(), # desliga aleatoriamente alguns neuronios durante o treino para evitar overfitting
                          nn.Linear(256 * 6 * 6, 4096), # entrada densa conectada com entrada de 9216 valores e 4096 neuronios
                          nn.ReLU(),
                          nn.Dropout(),
                          nn.Linear(4096, 4096),
                          nn.ReLU(),
                          nn.Linear(4096, num_classes)
                            )

    def forward(self, x):
         x = self.features(x)
         x = self.avgpool(x)
         x = torch.flatten(x, 1) # transforma tudo em um só vetor
         x = self.classifier(x)
         return x